## **Sparse Autoencoder**

A Sparse Autoencoder adds a sparsity constraint to the encoder's activation function. It forces the model to have few active neurons in the hidden layer, encouraging the network to learn more compact and meaningful features from the input data.

**Imports**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

**Data Loading**

In [ ]:
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Minimal Preprocessing**

In [ ]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

**Sparse Autoencoder Model**

In [ ]:
class SparseAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, sparsity_factor=0.05):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim)
        self.sparsity_factor = sparsity_factor
    
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = torch.sigmoid(self.decoder(encoded))
        return decoded

    def sparsity_penalty(self, encoded):
        rho = torch.mean(encoded, dim=0)
        return torch.sum(self.sparsity_factor * torch.abs(rho - 0.05))  # Encourages 5% sparsity

**Instantiate the model**

In [ ]:
model = SparseAutoencoder(input_dim=X_train.shape[1], hidden_dim=8)

**Train the Model**

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    optimizer.zero_grad()
    y_pred = model(torch.tensor(X_train, dtype=torch.float32))
    reconstruction_loss = loss_fn(y_pred, torch.tensor(X_train, dtype=torch.float32))
    sparsity_loss = model.sparsity_penalty(torch.relu(model.encoder(torch.tensor(X_train, dtype=torch.float32))))
    loss = reconstruction_loss + sparsity_loss
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/50], Loss: {loss.item():.4f}")